# Capstone Project - The Battle of Neighbourhoods 
# Housing Sales Prices & Venues Data Analysis of Barcelona



## 1. Introduction
This notebook is a data science project which is the basis of the Capstone Project of the IBM Data Science Professional Certificate, where the main task is compare neighborhoods or cities of our choice or to come up with a problem where Foursquare location data can be used to solve it. <br>

In my case, I have chosen to analyze The neighbourhoods of Barcelona, the city where I currently live in. With a population of 1.6 million within city limits in an area of only 102.2 Km2, Barcelona is one of the most densely populated cities in Europe. Barcelona is divided into  into 10 districts and there is also a subdivision of 73 neighbourhoods. This subdivision is what I am going to use this research project. <br>

When we think about <b>city residents</b> they prefer to choose areas with lower real state prices but at the same time they consider the venues, premises or services that are present in the neighbourhood. The aim of this study is to group similar neighbourhoods in terms of the density of venues and premises that are present on the area. If one specific city resident like a specific type of neoghbourhood because he likes the venues category that are on it, he can look for other neighbourhoods that belong to the same cluster and have lower prices. 

Another target audience of this research are <b>investors</b> that pretend to create a business, as they may want to select neighbourhoods where the business type that they want to invest is less intense and detect unattended areas.

Finally, this study might give some insights about factors that influence in the real state prices, which may be useful for <b>real state agents</b>. We will study if clusters with a similar density of venues and premises tend to have the same price in Barcelona, or on the contrary they are not a determinant factor. 

Barcelona has long been a leader in the movement of smart cities and it is part of a select group of pioneers in smart urban planning along with cities such as Singapore, Vienna, San Francisco and Copenhagen. As a part of this, it is very effective in collecting data and most of this data is available for free to general public. About most than 400 datasets are available in Barcelona's City Hall Open Data Service (https://opendata-ajuntament.barcelona.cat/en/), including information about venues located on each neighbourhood and purchase housing prices.


## 2. Data

### 2.1 Sources used in the project

The following sources has been used to carry out the project: <br>
- a) ED50 Administrative Divisions in 2010 from Carto BCN. Contains the code, description and geopraphical position of the center and the shape of each city neoghbouurhood and district. It is possible to download the information by creating a free account. This file will be used to get the geopraphical center of each neoghbourhood and also their boundaries to create maps. https://w20.bcn.cat/cartobcn/ <br>
<br>
- b) Purchase of registered properties of the city of Barcelona in 2018. Information about housing prices per square meter. Information will be used to detect correlations between neighbourhoods cluster ans average housing prices. https://opendata-ajuntament.barcelona.cat/data/en/dataset/est-mercat-immobiliari-compravenda-preu-total<br>
<br>
- c) Inventory of premises of the city of Barcelona. This it will be used to contrast results obtained by using Foursquare data. https://opendata-ajuntament.barcelona.cat/data/en/dataset/cens-activitats-comercials <br>
<br>
- d) Venues around a certain location by using a Foursquare API <br>

### 2.2 Analyze and visualiza data for the analysis

#### Import libraries

We import all the libraries that are going to be used in the whole notebook

In [2]:
import pandas as pd
import numpy as np

#!conda install -c conda-forge pyshp --yes
import shapefile #provides support to handle ESRI Shapefiles in pure Python.

#!conda install -c conda-forge utm --yes
import utm #allows to convert geospatial coordinates from UTM to GPS format

from json import dumps
#!conda install -c conda-forge unidecode --yes
import unidecode

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

import requests

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 
from sklearn import metrics
from scipy.spatial.distance import cdist

#### a) Administrative Divisions in 2010 from Carto BCN

Instead of using a post code approach, we have decided to use data from cartographic department of Barcelona Town Council with information of districts and neoghbourhoods, their shapes and boundaries. They can be downloaded for free from https://w20.bcn.cat/cartobcn/default.aspx?lang=en by creating a free account in ESRI format.<br>
<br>
ESRI format contains a shapefile (shp) which contains the shapes in vectorial format of the different neighbourhoods and districs of the city, and a database file (dfx) which contain some statistical data of every neighbourhood. They can be easily read with the Python library pyshp (https://pypi.org/project/pyshp/) <br>
<br>
The database contains the following fields: <br>
o District: District code  <br>
o NDistric: District name  <br>
o CBarri: Neighbourhood code  <br>
o NBarri: neighbourhood name  <br>
o Homes: Number of men  <br>
o Dones: Number of women. <br>
o Perimetr: Perimiter <br>
o Area: Surface in square meters  <br>
o Coord_X: X-axis centroid coordinates, in UTM format<br>
o Coord_Y: Y-axis centroid coordinates, in UTM format<br>
o Web1: Link to district's web page  <br>
o Web2: Link to city's annual stats  <br>
o Web3: Link to districts's annual stats  <br>
o Web4: Link to neoighboourhood's annual stats  <br>
<br>
A shapefile file with vectorial information of neighbourhood boundaries will be used to create city maps with Cloropeth library and Coord_X and Coord_Y files, as they define of the centroid of each neighbourhood, will be used to make queries with Foursquare API. However downloaded data use UTM format, and we need to convert it to GPS coordenates. Therefore we are going to transform the files.

Define define input and ouput filename parameters

In [3]:
raw_neighbourhood_file = 'raw_data/BCN_Barri_ED50_SHP'
processed_neighbourhood_file = 'processed_data/BCN_Barri_ED50_SHP'    
processed_neighbourhood_jsonfile = 'processed_data/BCN_Barri_ED50_SHP.json'    

In [4]:
#Using PyShp create a Reader object to access the data from the Barcelona Shapefile                
shp_r = shapefile.Reader(raw_neighbourhood_file)
shp_r.shapeType

5

In [5]:
#Create a Writer object to write data to as a new Shapefile
shp_w = shapefile.Writer(processed_neighbourhood_file)   

Set variables for access to the field information of both the original and new Shapefile. We visualize the name of the fields in Catalan that dataset contains

In [6]:
shp_fields_r = shp_r.fields
shp_fields_w = shp_w.fields
shp_fields_r 

[('DeletionFlag', 'C', 1, 0),
 ['C_Distri', 'C', 2, 0],
 ['N_Distri', 'C', 75, 0],
 ['C_Barri', 'C', 2, 0],
 ['N_Barri', 'C', 50, 0],
 ['Homes', 'N', 9, 0],
 ['Dones', 'N', 9, 0],
 ['Area', 'F', 19, 11],
 ['Perim', 'F', 19, 11],
 ['Coord_X', 'F', 19, 11],
 ['Coord_Y', 'F', 19, 11],
 ['WEB_1', 'C', 75, 0],
 ['WEB_2', 'C', 75, 0],
 ['WEB_3', 'C', 75, 0],
 ['WEB_4', 'C', 75, 0]]

We replicate the field structure from the original into the new. However, we will use this step to translate variable names to English and also change coords for GPS longitude and latitude. The ‘Deletion Flag’ as set in the Shapefile standard will be passed over (the tuple in the if statement).

In [7]:
for name in shp_fields_r:
    if type(name) == "tuple":
        continue
    elif name[0] == "C_Distri":
        args = name
        args[0] = 'C_Dist'
        shp_w.field(*args)
    elif name[0] == "N_Distri":
        args = name
        args[0] = 'N_Dist'
        shp_w.field(*args)
    elif name[0] == "C_Barri":
        args = name
        args[0] = 'C_Neigh'
        shp_w.field(*args)
    elif name[0] == "N_Barri":
        args = name
        args[0] = 'N_Neigh'
        shp_w.field(*args)
    elif name[0] == "Homes":
        args = name
        args[0] = 'Men' 
        shp_w.field(*args)
    elif name[0] == "Dones":
        args = name
        args[0] = 'Women' 
        shp_w.field(*args)
    elif name[0] == "Perim":
        args = name
        args[0] = 'Perimeter' 
        shp_w.field(*args)
    elif name[0] == "Coord_X":
        args = ['Longitude', 'F', 8, 16]
        shp_w.field(*args)
    elif name[0] == "Coord_Y":
        args = ['Latitude', 'F', 8, 16]     
        shp_w.field(*args)
    else:
        args = name
        shp_w.field(*args)
        
shp_w.fields

[('DeletionFlag', 'C', 1, 0),
 ('C_Dist', 'C', 2, 0),
 ('N_Dist', 'C', 75, 0),
 ('C_Neigh', 'C', 2, 0),
 ('N_Neigh', 'C', 50, 0),
 ('Men', 'N', 9, 0),
 ('Women', 'N', 9, 0),
 ('Area', 'F', 19, 11),
 ('Perimeter', 'F', 19, 11),
 ('Longitude', 'F', 8, 16),
 ('Latitude', 'F', 8, 16),
 ('WEB_1', 'C', 75, 0),
 ('WEB_2', 'C', 75, 0),
 ('WEB_3', 'C', 75, 0),
 ('WEB_4', 'C', 75, 0)]

Now we want to populate the fields with attribute information. In order to achive this, we create a variable to access the records of the original file.

In [8]:
shp_r.encoding = 'Latin-1' #Encoding for Western languages such as Catalan
records = shp_r.records()

In [10]:
records[0:4]

[Record #0: ['01', 'Ciutat Vella', '01', 'el Raval', 26553, 21850, 1098392.91379, 5557.3728782, 430732.006258, 4581363.15123, 'http://www.bcn.cat/ciutatvella', 'http://www.bcn.cat/estadistica/catala/dades/inf/anuari/index.htm', 'http://www.bcn.cat/estadistica/catala/dades/guiadt01/index.htm', 'http://www.bcn.cat/estadistica/catala/dades/inf/barris/a2011/barri1.pdf'],
 Record #1: ['01', 'Ciutat Vella', '02', 'el Barri Gòtic', 8368, 7508, 841905.077496, 5683.00485624, 431315.817811, 4581594.76566, 'http://www.bcn.cat/ciutatvella', 'http://www.bcn.cat/estadistica/catala/dades/inf/anuari/index.htm', 'http://www.bcn.cat/estadistica/catala/dades/guiadt01/index.htm', 'http://www.bcn.cat/estadistica/catala/dades/inf/barris/a2011/barri2.pdf'],
 Record #2: ['01', 'Ciutat Vella', '03', 'la Barceloneta', 7581, 7631, 1313867.98294, 13039.2667933, 432374.695771, 4581152.20728, 'http://www.bcn.cat/ciutatvella', 'http://www.bcn.cat/estadistica/catala/dades/inf/anuari/index.htm', 'http://www.bcn.cat/es

Copy the records from the original into the new. However, we are going to transform coordinates of the centroid from UTM format to GPS format. In this case it is needed to provide the UTM zone. In the case of Barcelona UTM zone is 31T. See more information in https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system .UTM library is used to carry out the transformation of coordinates.<br>
<br>
We will also use unicode libraty to remove accents and some special characters of Catalan language.

In [11]:
for row in records:
    args = row
    args[1] = unidecode.unidecode(args[1])
    args[3] = unidecode.unidecode(args[3])
    args[9],args[8] = utm.to_latlon(args[8], args[9], 31, 'T')
    shp_w.record(*args)

We need to access the geometry of the features in the original file so give yourself access to it.

In [12]:
geom = shp_r.shapes()

Now we loop through each feature in the original dataset, access every point that makes up the geometry, convert the coordinates for each point and re-assemble transformed geometry in the new Shapefile. The if statement will handle geometry with only one part making up the feature.

In [13]:
#This code is an adaption of how to reproject a Polygon Shapefile using PyShp 
#(https://glenbambrick.com/2016/01/24/reproject-shapefile/)
#but instead of using PyProj library we have used instead UTM libraty to transform coordinates

for feature in geom:
    # if there is only one part
    if len(feature.parts) == 1:
        # create empty list to store all the coordinates
        poly_list = []
        # get each coord that makes up the polygon
        for coords in feature.points:
            x, y = coords[0], coords[1]
            # tranform the coord
            #new_x, new_y = transform(input_projection, output_projection, x, y)
            new_y, new_x = utm.to_latlon(x, y, 31, 'T')
            # put the coord into a list structure
            poly_coord = (float(new_x), float(new_y))
            # append the coords to the polygon list
            poly_list.append(poly_coord)     
        # add the geometry to the shapefile.
        shp_w.poly([poly_list])
    else:
        # append the total amount of points to the end of the parts list
        feature.parts.append(len(feature.points))
        # enpty list to store all the parts that make up the complete feature
        poly_list = []
        # keep track of the part being added
        parts_counter = 0

        # while the parts_counter is less than the amount of parts
        while parts_counter < len(feature.parts) - 1:
            # keep track of the amount of points added to the feature
            coord_count = feature.parts[parts_counter]
            # number of points in each part
            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
            # create list to hold individual parts - these get added to poly_list[]
            part_list = []
            # cut off point for each part
            end_point = coord_count + no_of_points

            # loop through each part
            while coord_count < end_point:
                for coords in feature.points[coord_count:end_point]:
                    x, y = coords[0], coords[1]
                    # tranform the coord
                    #new_x, new_y = transform(input_projection, output_projection, x, y)
                    new_y, new_x = utm.to_latlon(x, y, 31, 'T')
                    # put the coord into a list structure
                    poly_coord = [float(new_x), float(new_y)]
                    # append the coords to the part list
                    part_list.append(poly_coord)
                    coord_count = coord_count + 1
            # append the part to the poly_list
            poly_list.append(part_list)
            parts_counter = parts_counter + 1
        # add the geometry to to new file
        shp_w.poly(poly_list)

We save the processed file

In [14]:
shp_w.close()

We create a processed GeoJSON file which it is going to be used to create map plots with Chloropeth library later

In [15]:
# read the shapefile
reader = shapefile.Reader(processed_neighbourhood_file)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    properties=atr, geometry=geom)) 

# write the GeoJSON file
geojson = open(processed_neighbourhood_jsonfile, "w")
geojson.write(dumps({"type": "FeatureCollection",\
"features": buffer}, indent=2) + "\n")
geojson.close()

Now we can load preprocessed neighbourhood file into a dataframe

In [17]:
#read file, parse out the records and shapes
shp_r = shapefile.Reader(processed_neighbourhood_file)
shp_r.encoding = 'Latin-1'

#grab the shapefile's field names (omit the first psuedo field)
fields = [x[0] for x in shp_r.fields][1:]
#records = sf.records()
records = [r[:] for r in shp_r.records()]
shps = [s.points for s in shp_r.shapes()]

#write the records into a dataframe
bcn_neigh_df = pd.DataFrame(columns=fields, data=records)

#add the coordinate data to a column called "coords"
bcn_neigh_df = bcn_neigh_df.assign(coords=shps)



Visualize the processed file with longitude and latitude information of each neighbourhood in the city. Fiels that are going to be used of this processed file are: <br>
- C_Neigh: Neighbourhood code <br>
- N_Neigh: Neighbourhood name <br>
- Longitude:- GPS Longitude of the neighbourhood centroid <br>
- Latitude: GPS Latitude of the neighbourhood centroid <br>

In [18]:
bcn_neigh_df.head(15)

,C_Dist,N_Dist,C_Neigh,N_Neigh,Men,Women,Area,Perimeter,Longitude,Latitude,WEB_1,WEB_2,WEB_3,WEB_4,coords
0,01,Ciutat Vella,01,el Raval,26553,21850,1.098393e+06,5557.372878,2.171593,41.38081,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.17114583251087, 41.387318300967955), (2.17..."
1,01,Ciutat Vella,02,el Barri Gotic,8368,7508,8.419051e+05,5683.004856,2.178549,41.38294,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.183565559937503, 41.38311899420626), (2.18..."
2,01,Ciutat Vella,03,la Barceloneta,7581,7631,1.313868e+06,13039.266793,2.191261,41.37905,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.20082586629123, 41.38678345631065), (2.200..."
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",11466,11390,1.114299e+06,4658.031512,2.184539,41.38864,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.1834921087968935, 41.39327449798472), (2.1..."
4,02,Eixample,05,el Fort Pienc,15039,16924,9.289010e+05,4175.971325,2.182589,41.39926,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.1834921087968935, 41.39327449798472), (2.1..."
5,02,Eixample,06,la Sagrada Familia,23686,27937,1.050991e+06,4336.621674,2.177686,41.40729,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.184464123030819, 41.40788069316526), (2.18..."
6,02,Eixample,07,la Dreta de l'Eixample,20240,23754,2.123405e+06,6460.307377,2.169300,41.39573,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.1744182656392472, 41.40183377858185), (2.1..."
7,02,Eixample,08,l'Antiga Esquerra de l'Eixample,19434,22727,1.234453e+06,4851.622805,2.156252,41.39120,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.165227982455392, 41.38815230194142), (2.16..."
8,02,Eixample,09,la Nova Esquerra de l'Eixample,27038,31186,1.337656e+06,4626.630284,2.150079,41.38490,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.159671850751981, 41.38397642110815), (2.15..."
9,02,Eixample,10,Sant Antoni,18134,20378,8.009851e+05,4711.253241,2.160452,41.38038,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.165948853147565, 41.387764793139524), (2.1..."


#### b) Housing prices 

Read CSV with housing price

In [3]:
raw_housing_price_file = 'raw_data/2018_comp_vend_preu.csv'
bcn_housing_price_df = pd.read_csv(raw_housing_price_file, dtype = {"Codi_Districte" : "str", "Codi_Barri" : "str" })
bcn_housing_price_df.head()

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Preu_mitja_habitatge,Valor
0,2018,1,Ciutat Vella,1,el Raval,Total. Milers d'euros,262.8
1,2018,1,Ciutat Vella,2,el Barri Gòtic,Total. Milers d'euros,541.6
2,2018,1,Ciutat Vella,3,la Barceloneta,Total. Milers d'euros,219.7
3,2018,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Total. Milers d'euros,336.9
4,2018,2,Eixample,5,el Fort Pienc,Total. Milers d'euros,408.1


We change column names to English

In [4]:
bcn_housing_price_df.rename(columns={"Any": "Year", "Codi_Districte": "C_Dist",  "Nom_Districte": "N_Dist", 
                                  "Codi_Barri": "C_Neigh", "Nom_Barri": "N_Neigh", "Valor": "Value", "Preu_mitja_habitatge" : "Concept"}, inplace = True)
bcn_housing_price_df.head()

,Year,C_Dist,N_Dist,C_Neigh,N_Neigh,Concept,Value
0,2018,1,Ciutat Vella,1,el Raval,Total. Milers d'euros,262.8
1,2018,1,Ciutat Vella,2,el Barri Gòtic,Total. Milers d'euros,541.6
2,2018,1,Ciutat Vella,3,la Barceloneta,Total. Milers d'euros,219.7
3,2018,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Total. Milers d'euros,336.9
4,2018,2,Eixample,5,el Fort Pienc,Total. Milers d'euros,408.1


We add 0 to numerical codes with only one character in order that match with neighbourhood file

In [5]:
bcn_housing_price_df['C_Dist'].replace('1','01', inplace=True)
bcn_housing_price_df['C_Dist'].replace('2','02', inplace=True)
bcn_housing_price_df['C_Dist'].replace('3','03', inplace=True)
bcn_housing_price_df['C_Dist'].replace('4','04', inplace=True)
bcn_housing_price_df['C_Dist'].replace('5','05', inplace=True)
bcn_housing_price_df['C_Dist'].replace('6','06', inplace=True)
bcn_housing_price_df['C_Dist'].replace('7','07', inplace=True)
bcn_housing_price_df['C_Dist'].replace('8','08', inplace=True)
bcn_housing_price_df['C_Dist'].replace('9','09', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('1','01', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('2','02', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('3','03', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('4','04', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('5','05', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('6','06', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('7','07', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('8','08', inplace=True)
bcn_housing_price_df['C_Neigh'].replace('9','09', inplace=True)
bcn_housing_price_df.head()

,Year,C_Dist,N_Dist,C_Neigh,N_Neigh,Concept,Value
0,2018,01,Ciutat Vella,01,el Raval,Total. Milers d'euros,262.8
1,2018,01,Ciutat Vella,02,el Barri Gòtic,Total. Milers d'euros,541.6
2,2018,01,Ciutat Vella,03,la Barceloneta,Total. Milers d'euros,219.7
3,2018,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",Total. Milers d'euros,336.9
4,2018,02,Eixample,05,el Fort Pienc,Total. Milers d'euros,408.1


In [6]:
bcn_housing_price_df.groupby(['Concept', 'Year']).count()

,,C_Dist,N_Dist,C_Neigh,N_Neigh,Value
Concept,Year,,,,,
Nou. Euros/m2 construït,2018,73,73,73,73,42
Nou. Milers d'euros,2018,73,73,73,73,42
Total. Euros/m2 construït,2018,73,73,73,73,73
Total. Milers d'euros,2018,73,73,73,73,73
Usat. Euros/m2 construït,2018,73,73,73,73,73
Usat. Milers d'euros,2018,73,73,73,73,73


We only want the average sales price of buildings sold (Preu_mitja_habitatge = 'Total Euros/m2 construit') of last year(2018). We are going to use the following files of this processed file: <br>
- C_Neigh: Neighbourhood code
- N_Neigh: Neighbourhood name
- AvgPRicem2 - Average price (EUR/m2) of purchased real state properties

In [7]:
bcn_housing_price_df = bcn_housing_price_df[bcn_housing_price_df['Concept']=='Total. Euros/m2 construït']
bcn_housing_price_df = bcn_housing_price_df[bcn_housing_price_df['Year']==2018]
bcn_housing_price_df.rename(columns={"Value": "AvgPricem2"}, inplace = True)
bcn_housing_price_df = bcn_housing_price_df[['C_Dist', 'N_Dist', 'C_Neigh', 'N_Neigh', 'AvgPricem2']]
bcn_housing_price_df.head()

,C_Dist,N_Dist,C_Neigh,N_Neigh,AvgPricem2
219,01,Ciutat Vella,01,el Raval,3969.8
220,01,Ciutat Vella,02,el Barri Gòtic,5162.7
221,01,Ciutat Vella,03,la Barceloneta,4905.4
222,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",5169.4
223,02,Eixample,05,el Fort Pienc,4650.6


#### c) Inventory of premises of the city of Barcelona.

Read CSV with the census of premises in Barcelona in 2016

In [9]:
raw_premises_file = 'raw_data/2016_cens_locals_plantabaixa.csv'
bcn_premises_df = pd.read_csv(raw_premises_file, dtype = {"Codi_Districte" : "str", "Codi_Barri" : "str", "SN_CCOMERC" : "str" })
bcn_premises_df.head()

,ID_BCN,ID_PRINCIP,N_PRINCIP,ID_SECTOR,N_SECTOR,ID_GRUPACT,N_GRUPACT,ID_ACT,N_ACT,N_LOCAL,...,Nom_Barri,Codi_Districte,N_DISTRI,N_EIX,SN_EIX,SEC_CENS,Y_UTM_ETRS,X_UTM_ETRS,LATITUD,LONGITUD
0,49,1,Actiu,1,Comerç al detall,4,Equipament personal,4003,Vestir,INSIDE,...,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580750.861,431675.395,41.375377,2.182944
1,50,1,Actiu,2,Serveis,14,"Restaurants, bars i hotels (Inclòs hostals, pe...",14000,Serveis de menjar i begudes,KURTZ & GUT,...,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580752.167,431687.786,41.375390,2.183092
2,51,1,Actiu,1,Comerç al detall,4,Equipament personal,4003,Vestir,SPRINGFIELD,...,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580753.613,431698.242,41.375404,2.183217
3,52,1,Actiu,1,Comerç al detall,4,Equipament personal,4000,Calçat i pell,CASAS KIDS,...,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580755.360,431709.717,41.375420,2.183354
4,53,1,Actiu,2,Serveis,14,"Restaurants, bars i hotels (Inclòs hostals, pe...",14000,Serveis de menjar i begudes,BARITIMO LOUGE CLUB,...,el Barri Gòtic,1,Ciutat Vella,NaN,0,1030,4580755.700,431723.319,41.375425,2.183517


We translate the name of some columns to English and take a look of some rows of the file. This census contains all the premises with their name (N_Local) and the neighbourhood where they are located (C_Neigh, N_Neigh)

In [10]:
bcn_premises_df.rename(columns={"ID_GRUPACT": "C_ActGrp", "N_GRUPACT": "N_ActGrp", "ID_ACT": "C_Act", "ID_ACT": "C_Act", "N_ACT": "N_Act","N_LOCAL" : "N_Local",
                                     "Codi_Districte": "C_Dist", "N_DISTRI": "N_Dist", "Codi_Barri": "C_Neigh", "Nom_Barri": "N_Neigh", "LATITUD": "Latitude", "LONGITUD" : "Longitude"}, inplace = True)
bcn_premises_df = bcn_premises_df[bcn_premises_df['ID_PRINCIP']==1]
bcn_premises_df = bcn_premises_df[["C_Act","N_Act","N_Local","C_Neigh","N_Neigh","C_Dist","N_Dist","Latitude","Longitude"]]
bcn_premises_df.head(15)

,C_Act,N_Act,N_Local,C_Neigh,N_Neigh,C_Dist,N_Dist,Latitude,Longitude
0,4003,Vestir,INSIDE,2,el Barri Gòtic,1,Ciutat Vella,41.375377,2.182944
1,14000,Serveis de menjar i begudes,KURTZ & GUT,2,el Barri Gòtic,1,Ciutat Vella,41.375390,2.183092
2,4003,Vestir,SPRINGFIELD,2,el Barri Gòtic,1,Ciutat Vella,41.375404,2.183217
3,4000,Calçat i pell,CASAS KIDS,2,el Barri Gòtic,1,Ciutat Vella,41.375420,2.183354
4,14000,Serveis de menjar i begudes,BARITIMO LOUGE CLUB,2,el Barri Gòtic,1,Ciutat Vella,41.375425,2.183517


We add 0 to numerical codes with only one character in order that match with neighbourhood file

In [11]:
bcn_premises_df['C_Dist'].replace('1','01', inplace=True)
bcn_premises_df['C_Dist'].replace('2','02', inplace=True)
bcn_premises_df['C_Dist'].replace('3','03', inplace=True)
bcn_premises_df['C_Dist'].replace('4','04', inplace=True)
bcn_premises_df['C_Dist'].replace('5','05', inplace=True)
bcn_premises_df['C_Dist'].replace('6','06', inplace=True)
bcn_premises_df['C_Dist'].replace('7','07', inplace=True)
bcn_premises_df['C_Dist'].replace('8','08', inplace=True)
bcn_premises_df['C_Dist'].replace('9','09', inplace=True)
bcn_premises_df['C_Neigh'].replace('1','01', inplace=True)
bcn_premises_df['C_Neigh'].replace('2','02', inplace=True)
bcn_premises_df['C_Neigh'].replace('3','03', inplace=True)
bcn_premises_df['C_Neigh'].replace('4','04', inplace=True)
bcn_premises_df['C_Neigh'].replace('5','05', inplace=True)
bcn_premises_df['C_Neigh'].replace('6','06', inplace=True)
bcn_premises_df['C_Neigh'].replace('7','07', inplace=True)
bcn_premises_df['C_Neigh'].replace('8','08', inplace=True)
bcn_premises_df['C_Neigh'].replace('9','09', inplace=True)
bcn_premises_df.head()

,C_Act,N_Act,N_Local,C_Neigh,N_Neigh,C_Dist,N_Dist,Latitude,Longitude
0,4003,Vestir,INSIDE,02,el Barri Gòtic,01,Ciutat Vella,41.375377,2.182944
1,14000,Serveis de menjar i begudes,KURTZ & GUT,02,el Barri Gòtic,01,Ciutat Vella,41.375390,2.183092
2,4003,Vestir,SPRINGFIELD,02,el Barri Gòtic,01,Ciutat Vella,41.375404,2.183217
3,4000,Calçat i pell,CASAS KIDS,02,el Barri Gòtic,01,Ciutat Vella,41.375420,2.183354
4,14000,Serveis de menjar i begudes,BARITIMO LOUGE CLUB,02,el Barri Gòtic,01,Ciutat Vella,41.375425,2.183517


We can count the number of premises that are on each category. N_Act contains the category of the premise in Catalan description (unfornately datasets can not be obtained in English). As we can see we have a total of 70 distinct categories of premises.

In [15]:
bcn_premises_df.groupby(['N_Act']).count()

,C_Act,N_Local,C_Neigh,N_Neigh,C_Dist,N_Dist,Latitude,Longitude
N_Act,,,,,,,,
Activitats de la construcció,1827,1827,1827,1827,1827,1827,1827,1827
Activitats de transport i emmagatzematge,2008,2008,2008,2008,2008,2008,2008,2008
Activitats immobiliàries,805,805,805,805,805,805,805,805
Activitats industrials,341,341,341,341,341,341,341,341
Administració,311,311,311,311,311,311,311,311
Agències de viatge,324,324,324,324,324,324,324,324
Altres,2906,2906,2906,2906,2906,2906,2906,2906
Aparells domèstics,191,191,191,191,191,191,191,191
Arranjaments,552,552,552,552,552,552,552,552


Count the total number of disctint categories present in the dataframe

In [16]:
print('There are {} uniques categories.'.format(len(bcn_premises_df['N_Act'].unique())))

There are 61 uniques categories.


Count the total number of premises in the dataframe

In [19]:
print('There are {} premises.'.format(bcn_premises_df.shape[0]))

There are 60265 premises.


#### d) Foursquare data of venues

Use the Foursquare API to explore and segment neighbourhoods. API credentials are defined in a hidden cell for privacy reasons. <br><br>

CLIENT_ID = 'portion hidden from view' # your Foursquare ID <br>
CLIENT_SECRET = 'portion hidden from view' # your Foursquare Secret <br>
VERSION = '20180604' <br>
print('Your credentails:') <br>
print('CLIENT_ID: ' + CLIENT_ID) <br>
print('CLIENT_SECRET:' + CLIENT_SECRET) <br>

In [1]:
# The code was removed by Watson Studio for sharing.

Create a function to explore the nearby venues in Foursqaure for all the neighborhoods in Barcelona within a radius of 500 meters. A maximum limit of 200 venues per neighbourhood will be setup.-

In [34]:
# type your answer here
LIMIT = 200 # limit of number of venues returned by Foursquare API

def getNearbyVenues(codes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, name, lat, lng in zip(codes, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['C_Neigh', 
                  'N_Neigh',                
                  'Neigh_Lat', 
                  'Neigh_Long', 
                  'Venue', 
                  'Venue_Lat', 
                  'Venue_Long', 
                  'Venue_Category']
    
    return(nearby_venues)

Get all the venues into a dataframe

In [35]:
bcn_venues = getNearbyVenues(codes=bcn_neigh_df['C_Neigh'],
                                   names=bcn_neigh_df['N_Neigh'],
                                   latitudes=bcn_neigh_df['Latitude'],
                                   longitudes=bcn_neigh_df['Longitude']
                                  )

el Raval
el Barri Gotic
la Barceloneta
Sant Pere, Santa Caterina i la Ribera
el Fort Pienc
la Sagrada Familia
la Dreta de l'Eixample
l'Antiga Esquerra de l'Eixample
la Nova Esquerra de l'Eixample
Sant Antoni
el Poble Sec
la Marina del Prat Vermell
la Marina de Port
la Font de la Guatlla
Hostafrancs
la Bordeta
Sants - Badal
Sants
les Corts
la Maternitat i Sant Ramon
Pedralbes
Vallvidrera, el Tibidabo i les Planes
Sarria
les Tres Torres
Sant Gervasi - la Bonanova
Sant Gervasi - Galvany
el Putxet i el Farro
Vallcarca i els Penitents
el Coll
la Salut
la Vila de Gracia
el Camp d'en Grassot i Gracia Nova
el Baix Guinardo
Can Baro
el Guinardo
la Font d'en Fargues
el Carmel
la Teixonera
Sant Genis dels Agudells
Montbau
la Vall d'Hebron
la Clota
Horta
Vilapicina i la Torre Llobeta
Porta
el Turo de la Peira
Can Peguera
la Guineueta
Canyelles
les Roquetes
Verdun
la Prosperitat
la Trinitat Nova
Torre Baro
Ciutat Meridiana
Vallbona
la Trinitat Vella
Baro de Viver
el Bon Pastor
Sant Andreu
la Sagrer

In [36]:
bcn_4square_venues_file = 'processed_data/BCN_FourSquareVenues.csv'
bcn_venues.to_csv(bcn_4square_venues_file)

In [21]:
bcn_4square_venues_file = 'processed_data/BCN_FourSquareVenues.csv'
bcn_venues = pd.read_csv(bcn_4square_venues_file, dtype = {"C_Neigh" : "str", "C_Dist" : "str"})

Check the size of the resulting dataframe

In [22]:
print(bcn_venues.shape)
bcn_venues.head()

(2878, 9)


,Unnamed: 0,C_Neigh,N_Neigh,Neigh_Lat,Neigh_Long,Venue,Venue_Lat,Venue_Long,Venue_Category
0,0,01,el Raval,41.38081,2.171593,Mercat de Sant Josep - La Boqueria,41.381959,2.172011,Market
1,1,01,el Raval,41.38081,2.171593,Kӕlderkold,41.381723,2.173348,Beer Bar
2,2,01,el Raval,41.38081,2.171593,El Quim de la Boqueria,41.381908,2.171949,Tapas Restaurant
3,3,01,el Raval,41.38081,2.171593,Liceu Opera Barcelona,41.380713,2.173606,Opera House
4,4,01,el Raval,41.38081,2.171593,Miró Mosaic on the Rambla,41.381158,2.173280,Plaza


check how many venues were returned for each neighborhood

In [23]:
bcn_venues.groupby('N_Neigh').count()

,Unnamed: 0,C_Neigh,Neigh_Lat,Neigh_Long,Venue,Venue_Lat,Venue_Long,Venue_Category
N_Neigh,,,,,,,,
Baro de Viver,2,2,2,2,2,2,2,2
Can Baro,13,13,13,13,13,13,13,13
Can Peguera,17,17,17,17,17,17,17,17
Canyelles,4,4,4,4,4,4,4,4
Ciutat Meridiana,5,5,5,5,5,5,5,5
Diagonal Mar i el Front Maritim del Poblenou,64,64,64,64,64,64,64,64
Horta,4,4,4,4,4,4,4,4
Hostafrancs,42,42,42,42,42,42,42,42
Montbau,4,4,4,4,4,4,4,4


We can count the number of venues that are on each category. 

In [24]:
bcn_venues.groupby(['Venue_Category']).count()

,Unnamed: 0,C_Neigh,N_Neigh,Neigh_Lat,Neigh_Long,Venue,Venue_Lat,Venue_Long
Venue_Category,,,,,,,,
Accessories Store,4,4,4,4,4,4,4,4
African Restaurant,1,1,1,1,1,1,1,1
American Restaurant,4,4,4,4,4,4,4,4
Amphitheater,1,1,1,1,1,1,1,1
Antique Shop,1,1,1,1,1,1,1,1
Arcade,2,2,2,2,2,2,2,2
Arepa Restaurant,2,2,2,2,2,2,2,2
Argentinian Restaurant,12,12,12,12,12,12,12,12
Art Gallery,10,10,10,10,10,10,10,10


Find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(bcn_venues['Venue_Category'].unique())))

There are 272 uniques categories.


Count the total number of premises in the dataframe

In [26]:
print('There are {} venues.'.format(bcn_venues.shape[0]))

There are 2878 venues.


#### 3) Comparison between Forusquare and the official census of premises
As we can see Foursquare venue data uses a much wider category definition (273) vs the 61 categories of the premises census (70). On the other hand the census of premises proveides with 60265 premises grouped in 70 categories. <br>